In [2]:
using Pkg
Pkg.activate("Phase3_projet")
Pkg.add(["LinearAlgebra", "SparseArrays", "Krylov", "BenchmarkTools", "SuiteSparseMatrixCollection", "MatrixMarket"])
using LinearAlgebra, SparseArrays, Krylov, BenchmarkTools, SuiteSparseMatrixCollection, MatrixMarket, Random, Plots

  Activating project at `c:\Users\ulric\Projet_MTH8211\Phase3_projet`
   Resolving package versions...
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase3_projet\Project.toml`
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase3_projet\Manifest.toml`


In [3]:
function badly_conditioned_rectangular_matrix(m, n, kappa)
    U, _ = qr(randn(m, n))
    V, _ = qr(randn(n, n))
    s = range(1.0, 1.0/kappa, length=n)
    S = Diagonal(s)
    A = U * S * V'
    return Matrix(A)
end

function badly_conditioned_underdetermined_matrix(m, n, kappa)
    U, _ = qr(randn(n, m))
    V, _ = qr(randn(m, m))
    s = range(1.0, 1.0 / kappa, length=m)
    S = Diagonal(s)
    A_tall = U * S * V'
    return Matrix(A_tall')[1:m, 1:n]
end

function lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n) 
    G = randn(s, m)
    Ã = G * A
    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃.> 1e-12)
    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:,1:r]
    N = V_r * Σinv
    AN = A * N
    ŷ, histo = lsqr(AN, b; atol=tol, btol=tol, itmax=itmax, history=true)

    x̂ = N * ŷ
    return x̂, histo
end

function lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * m)
    G = randn(n, s)
    Ã = A * G
    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)
    U_r = Ũ[:,1:r]
    Σ_r = Σ̃[1:r]
    Σinv = Diagonal(1.0 ./ Σ_r)
    M = U_r * Σinv      
    Mt = M'
    At_pre = Mt * A    
    bt_pre = Mt * b     

    x̂, histo = lsqr(At_pre, bt_pre; atol=tol, btol=tol, itmax=itmax, history=true)

    return x̂, histo
end

lsrn_lsqr_underdetermined (generic function with 1 method)

# Conditionnement après préconditionnement

In [4]:
function precondition_over(A; γ=2.0)
    m, n = size(A)
    s = ceil(Int, γ * n)
    G = randn(s, m)
    Ã = G * A
    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)
    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:, 1:r]
    N = V_r * Σinv
    AN = A * N
    return AN
end

function precondition_under(A; γ=2.0)
    m, n = size(A)
    s = ceil(Int, γ * m)
    G = randn(n, s)
    Ã = A * G
    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)
    U_r = Ũ[:, 1:r]
    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    M = U_r * Σinv
    Mt = M'
    At_pre = Mt * A
    return At_pre
end

precondition_under (generic function with 1 method)

In [14]:
kappas = 10 .^ range(1, 7, length=9)
dims = [(100, 100), (1000, 100), (10000, 100)]
dims_under = [(100, 100), (100, 1000), (100, 10000)]

results = Dict()

for (m,n) in dims
    ys = Float64[]
    for κ in kappas
        A = badly_conditioned_rectangular_matrix(m, n, κ)
        AN = precondition_over(A)
        push!(ys, cond(AN))
    end
    results[("over", m, n)] = ys
end

for (m,n) in dims_under
    ys = Float64[]
    for κ in kappas
        A = badly_conditioned_underdetermined_matrix(m, n, κ)
        AN = precondition_under(A)
        push!(ys, cond(AN))
    end
    results[("under", m, n)] = ys
end

plt = plot(xscale=:log10,
    xlabel="Cond(A)", ylabel="Cond(A preconditionnée)",
    title="Amélioration du conditionnement")

for (m,n) in dims
    plot!(plt, kappas, results[("over", m, n)],
        lw=2, label="Surdéterminé m=$m, n=$n")
end

for (m,n) in dims_under
    plot!(plt, kappas, results[("under", m, n)],
        lw=2, linestyle=:dash, label="Sous-déterminé m=$m, n=$n")
end
savefig(plt, "Fig/cond_improvement.png")
savefig(plt, "Fig/cond_improvement.pdf")

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\cond_improvement.pdf"

# Système sur-déterminé

### ColorMap de Gain en focntion du conditionnement allant de 1 à 1000000 et ratio m/n allant de 1 à 1000 (avec m0=n0=1000)

In [15]:
ms = Int.(round.(vec(10 .^ range(3, 6, length=4))))
n = 1000
kappa_vals = vec(10 .^ range(0, 6, length=7))
gain = zeros(length(kappa_vals),length(ms))
for (j, m) in enumerate(ms)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_rectangular_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("m           : $(m)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =heatmap(ms, kappa_vals, gain;
    xscale=:log10,
    yscale=:log10,
    xlabel="Ratio m/n",
    ylabel="Conditionnement",
    color=:viridis,
    title="Gain",
    clim=(-1, 1)
)
savefig(plt,"Fig/Gain_VS_Condi-ratio_surdet_1000.png")
savefig(plt,"Fig/Gain_VS_Condi-ratio_surdet_1000.pdf")

m           : 1000
Kappa           : 1.0
  
m           : 1000
Kappa           : 10.0
  
m           : 1000
Kappa           : 100.0
  
m           : 1000
Kappa           : 1000.0
  
m           : 1000
Kappa           : 10000.0
  
m           : 1000
Kappa           : 100000.0
  
m           : 1000
Kappa           : 1.0e6
  
m           : 10000
Kappa           : 1.0
  
m           : 10000
Kappa           : 10.0
  
m           : 10000
Kappa           : 100.0
  
m           : 10000
Kappa           : 1000.0
  
m           : 10000
Kappa           : 10000.0
  
m           : 10000
Kappa           : 100000.0
  
m           : 10000
Kappa           : 1.0e6
  
m           : 100000
Kappa           : 1.0
  
m           : 100000
Kappa           : 10.0
  
m           : 100000
Kappa           : 100.0
  
m           : 100000
Kappa           : 1000.0
  
m           : 100000
Kappa           : 10000.0
  
m           : 100000
Kappa           : 100000.0
  
m           : 100000
Kappa           : 1.0e6
  
m   

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_Condi-ratio_surdet_1000.pdf"

### ColorMap de Gain en focntion du conditionnement allant de 1 à 1000000 et ratio m/n allant de 1 à 1000 (avec m0=n0=100)

In [16]:
ms = Int.(round.(vec(10 .^ range(2, 5, length=4))))
n = 100
kappa_vals = vec(10 .^ range(0, 6, length=7))
gain = zeros(length(kappa_vals),length(ms))
for (j, m) in enumerate(ms)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_rectangular_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("m           : $(m)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =heatmap(ms, kappa_vals, gain;
    xscale=:log10,
    yscale=:log10,
    xlabel="Ratio m/n",
    ylabel="Conditionnement",
    color=:viridis,
    title="Gain",
    clim=(-1, 1)
)
savefig(plt,"Fig/Gain_VS_Condi-ratio_surdet_100.png")
savefig(plt,"Fig/Gain_VS_Condi-ratio_surdet_100.pdf")

m           : 100
Kappa           : 1.0
  
m           : 100
Kappa           : 10.0
  
m           : 100
Kappa           : 100.0
  
m           : 100
Kappa           : 1000.0
  
m           : 100
Kappa           : 10000.0
  
m           : 100
Kappa           : 100000.0
  
m           : 100
Kappa           : 1.0e6
  
m           : 1000
Kappa           : 1.0
  
m           : 1000
Kappa           : 10.0
  
m           : 1000
Kappa           : 100.0
  
m           : 1000
Kappa           : 1000.0
  
m           : 1000
Kappa           : 10000.0
  
m           : 1000
Kappa           : 100000.0
  
m           : 1000
Kappa           : 1.0e6
  
m           : 10000
Kappa           : 1.0
  
m           : 10000
Kappa           : 10.0
  
m           : 10000
Kappa           : 100.0
  
m           : 10000
Kappa           : 1000.0
  
m           : 10000
Kappa           : 10000.0
  
m           : 10000
Kappa           : 100000.0
  
m           : 10000
Kappa           : 1.0e6
  
m           : 100000
Kapp

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_Condi-ratio_surdet_100.pdf"

### ColorMap de Gain en focntion du conditionnement allant de 1 à 1000000 et ratio m/n allant de 1 à 1000 (avec m0=n0=10)

In [17]:
ms = Int.(round.(vec(10 .^ range(1, 4, length=4))))
n = 10
kappa_vals = vec(10 .^ range(0, 6, length=7))
gain = zeros(length(kappa_vals),length(ms))
for (j, m) in enumerate(ms)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_rectangular_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("m           : $(m)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =heatmap(ms, kappa_vals, gain;
    xscale=:log10,
    yscale=:log10,
    xlabel="Ratio m/n",
    ylabel="Conditionnement",
    color=:viridis,
    title="Gain",
    clim=(-1, 1)
)
savefig(plt,"Fig/Gain_VS_Condi-ratio_surdet_10.png")
savefig(plt,"Fig/Gain_VS_Condi-ratio_surdet_10.pdf")

m           : 10
Kappa           : 1.0
  
m           : 10
Kappa           : 10.0
  
m           : 10
Kappa           : 100.0
  
m           : 10
Kappa           : 1000.0
  
m           : 10
Kappa           : 10000.0
  
m           : 10
Kappa           : 100000.0
  
m           : 10
Kappa           : 1.0e6
  
m           : 100
Kappa           : 1.0
  
m           : 100
Kappa           : 10.0
  
m           : 100
Kappa           : 100.0
  
m           : 100
Kappa           : 1000.0
  
m           : 100
Kappa           : 10000.0
  
m           : 100
Kappa           : 100000.0
  
m           : 100
Kappa           : 1.0e6
  
m           : 1000
Kappa           : 1.0
  
m           : 1000
Kappa           : 10.0
  
m           : 1000
Kappa           : 100.0
  
m           : 1000
Kappa           : 1000.0
  
m           : 1000
Kappa           : 10000.0
  
m           : 1000
Kappa           : 100000.0
  
m           : 1000
Kappa           : 1.0e6
  
m           : 10000
Kappa           : 1.0
  
m

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_Condi-ratio_surdet_10.pdf"

### Gain vs ratio m/n

In [18]:
ms = Int.(round.(vec(10 .^ range(2, 5, length=500))))
n = 100
kappa_vals = 10
gain = zeros(length(kappa_vals),length(ms))
for (j, m) in enumerate(ms)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_rectangular_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("m           : $(m)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =plot(ms, vec(gain);
    xscale=:log10,
    xlabel="Ratio m/n",
    ylabel="Gain",
    color=:viridis,
    title="Gain en fonction du ratio m/n",
)
savefig(plt,"Fig/Gain_VS_ratio_surdet_100.png")
savefig(plt,"Fig/Gain_VS_ratio_surdet_100.pdf")

m           : 100
Kappa           : 10
  
m           : 101
Kappa           : 10
  
m           : 103
Kappa           : 10
  
m           : 104
Kappa           : 10
  
m           : 106
Kappa           : 10
  
m           : 107
Kappa           : 10
  
m           : 109
Kappa           : 10
  
m           : 110
Kappa           : 10
  
m           : 112
Kappa           : 10
  
m           : 113
Kappa           : 10
  
m           : 115
Kappa           : 10
  
m           : 116
Kappa           : 10
  
m           : 118
Kappa           : 10
  
m           : 120
Kappa           : 10
  
m           : 121
Kappa           : 10
  
m           : 123
Kappa           : 10
  
m           : 125
Kappa           : 10
  
m           : 127
Kappa           : 10
  
m           : 128
Kappa           : 10
  
m           : 130
Kappa           : 10
  
m           : 132
Kappa           : 10
  
m           : 134
Kappa           : 10
  
m           : 136
Kappa           : 10
  
m           : 137
Kappa           

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_ratio_surdet_100.pdf"

### Gain vs conditionnement

In [19]:
ms = 10000
n = 100
kappa_vals = vec(10 .^ range(0, 2, length=200))
gain = zeros(length(kappa_vals),length(ms))
for (j, m) in enumerate(ms)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_rectangular_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("m           : $(m)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =plot(kappa_vals, vec(gain);
    xscale=:log10,
    xlabel="Conditionnement",
    ylabel="Gain",
    color=:viridis,
    title="Gain en fonction du conditionnement",
)
savefig(plt,"Fig/Gain_VS_condi_surdet_100-10000.png")
savefig(plt,"Fig/Gain_VS_condi_surdet_100-10000.pdf")


m           : 10000
Kappa           : 1.0
  
m           : 10000
Kappa           : 1.0234114021054532
  
m           : 10000
Kappa           : 1.0473708979594496
  
m           : 10000
Kappa           : 1.0718913192051278
  
m           : 10000
Kappa           : 1.0969857978923836
  
m           : 10000
Kappa           : 1.1226677735108137
  
m           : 10000
Kappa           : 1.1489510001873091
  
m           : 10000
Kappa           : 1.1758495540521567
  
m           : 10000
Kappa           : 1.2033778407775895
  
m           : 10000
Kappa           : 1.2315506032928256
  
m           : 10000
Kappa           : 1.2603829296797275
  
m           : 10000
Kappa           : 1.2898902612533087
  
m           : 10000
Kappa           : 1.3200884008314178
  
m           : 10000
Kappa           : 1.3509935211980268
  
m           : 10000
Kappa           : 1.3826221737646558
  
m           : 10000
Kappa           : 1.414991297434576
  
m           : 10000
Kappa           : 1.4481182276745337

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_condi_surdet_100-10000.pdf"

# Système sous-déterminé

### ColorMap de Gain en focntion du conditionnement allant de 1 à 1000000 et ratio n/m allant de 1 à 1000 (avec m0=n0=10)

In [20]:
m = 10
ns = Int.(round.(vec(10 .^ range(1, 4, length=4))))
kappa_vals = vec(10 .^ range(0, 6, length=7))
gain = zeros(length(kappa_vals),length(ns))
for (j, n) in enumerate(ns)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_underdetermined_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("n           : $(n)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =heatmap(ns, kappa_vals, gain;
    xscale=:log10,
    yscale=:log10,
    xlabel="Nombre de colonnes",
    ylabel="Conditionnement",
    color=:viridis,
    title="Niter/Niter_LSRN",
    clim=(-1, 1)
)
savefig(plt,"Fig/Gain_VS_Condi-ratio_sousdet_10.png")
savefig(plt,"Fig/Gain_VS_Condi-ratio_sousdet_10.pdf")

n           : 10
Kappa           : 1.0
  
n           : 10
Kappa           : 10.0
  
n           : 10
Kappa           : 100.0
  
n           : 10
Kappa           : 1000.0
  
n           : 10
Kappa           : 10000.0
  
n           : 10
Kappa           : 100000.0
  
n           : 10
Kappa           : 1.0e6
  
n           : 100
Kappa           : 1.0
  
n           : 100
Kappa           : 10.0
  
n           : 100
Kappa           : 100.0
  
n           : 100
Kappa           : 1000.0
  
n           : 100
Kappa           : 10000.0
  
n           : 100
Kappa           : 100000.0
  
n           : 100
Kappa           : 1.0e6
  
n           : 1000
Kappa           : 1.0
  
n           : 1000
Kappa           : 10.0
  
n           : 1000
Kappa           : 100.0
  
n           : 1000
Kappa           : 1000.0
  
n           : 1000
Kappa           : 10000.0
  
n           : 1000
Kappa           : 100000.0
  
n           : 1000
Kappa           : 1.0e6
  
n           : 10000
Kappa           : 1.0
  
n

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_Condi-ratio_sousdet_10.pdf"

### ColorMap de Gain en focntion du conditionnement allant de 1 à 1000000 et ratio n/m allant de 1 à 1000 (avec m0=n0=100)

In [21]:
m = 100
ns = Int.(round.(vec(10 .^ range(2, 5, length=4))))
kappa_vals = vec(10 .^ range(0, 6, length=7))
gain = zeros(length(kappa_vals),length(ns))
for (j, n) in enumerate(ns)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_underdetermined_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("n           : $(n)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =heatmap(ns, kappa_vals, gain;
    xscale=:log10,
    yscale=:log10,
    xlabel="Nombre de colonnes",
    ylabel="Conditionnement",
    color=:viridis,
    title="Niter/Niter_LSRN",
    clim=(-1, 1)
)
savefig(plt,"Fig/Gain_VS_Condi-ratio_sousdet_100.png")
savefig(plt,"Fig/Gain_VS_Condi-ratio_sousdet_100.pdf")

n           : 100
Kappa           : 1.0
  
n           : 100
Kappa           : 10.0
  
n           : 100
Kappa           : 100.0
  
n           : 100
Kappa           : 1000.0
  
n           : 100
Kappa           : 10000.0
  
n           : 100
Kappa           : 100000.0
  
n           : 100
Kappa           : 1.0e6
  
n           : 1000
Kappa           : 1.0
  
n           : 1000
Kappa           : 10.0
  
n           : 1000
Kappa           : 100.0
  
n           : 1000
Kappa           : 1000.0
  
n           : 1000
Kappa           : 10000.0
  
n           : 1000
Kappa           : 100000.0
  
n           : 1000
Kappa           : 1.0e6
  
n           : 10000
Kappa           : 1.0
  
n           : 10000
Kappa           : 10.0
  
n           : 10000
Kappa           : 100.0
  
n           : 10000
Kappa           : 1000.0
  
n           : 10000
Kappa           : 10000.0
  
n           : 10000
Kappa           : 100000.0
  
n           : 10000
Kappa           : 1.0e6
  
n           : 100000
Kapp

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_Condi-ratio_sousdet_100.pdf"

### ColorMap de Gain en focntion du conditionnement allant de 1 à 1000000 et ratio n/m allant de 1 à 1000 (avec m0=n0=1000)

In [22]:
m = 1000
ns = Int.(round.(vec(10 .^ range(3, 6, length=4))))
kappa_vals = vec(10 .^ range(0, 6, length=7))
gain = zeros(length(kappa_vals),length(ns))
for (j, n) in enumerate(ns)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_underdetermined_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("n           : $(n)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =heatmap(ns, kappa_vals, gain;
    xscale=:log10,
    yscale=:log10,
    xlabel="Nombre de colonnes",
    ylabel="Conditionnement",
    color=:viridis,
    title="Niter/Niter_LSRN",
    clim=(-1, 1)
)
savefig(plt,"Fig/Gain_VS_Condi-ratio_sousdet_1000.png")
savefig(plt,"Fig/Gain_VS_Condi-ratio_sousdet_1000.pdf")

n           : 1000
Kappa           : 1.0
  
n           : 1000
Kappa           : 10.0
  
n           : 1000
Kappa           : 100.0
  
n           : 1000
Kappa           : 1000.0
  
n           : 1000
Kappa           : 10000.0
  
n           : 1000
Kappa           : 100000.0
  
n           : 1000
Kappa           : 1.0e6
  
n           : 10000
Kappa           : 1.0
  
n           : 10000
Kappa           : 10.0
  
n           : 10000
Kappa           : 100.0
  
n           : 10000
Kappa           : 1000.0
  
n           : 10000
Kappa           : 10000.0
  
n           : 10000
Kappa           : 100000.0
  
n           : 10000
Kappa           : 1.0e6
  
n           : 100000
Kappa           : 1.0
  
n           : 100000
Kappa           : 10.0
  
n           : 100000
Kappa           : 100.0
  
n           : 100000
Kappa           : 1000.0
  
n           : 100000
Kappa           : 10000.0
  
n           : 100000
Kappa           : 100000.0
  
n           : 100000
Kappa           : 1.0e6
  
n   

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_Condi-ratio_sousdet_1000.pdf"

### Gain vs ratio m/n

In [5]:
m = 100
ns = Int.(round.(vec(10 .^ range(2, 5, length=500))))
kappa_vals = 10
gain = zeros(length(kappa_vals),length(ns))
for (j, n) in enumerate(ns)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_underdetermined_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("n           : $(n)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =plot(ns, vec(gain);
    xscale=:log10,
    xlabel="Ratio n/m",
    ylabel="Gain",
    color=:viridis,
    title="Gain en fonction du ratio n/m",
)
savefig(plt,"Fig/Gain_VS_ratio_sousdet_100.png")
savefig(plt,"Fig/Gain_VS_ratio_sousdet_100.pdf")

n           : 100
Kappa           : 10
  
n           : 101
Kappa           : 10
  
n           : 103
Kappa           : 10
  
n           : 104
Kappa           : 10
  
n           : 106
Kappa           : 10
  
n           : 107
Kappa           : 10
  
n           : 109
Kappa           : 10
  
n           : 110
Kappa           : 10
  
n           : 112
Kappa           : 10
  
n           : 113
Kappa           : 10
  
n           : 115
Kappa           : 10
  
n           : 116
Kappa           : 10
  
n           : 118
Kappa           : 10
  
n           : 120
Kappa           : 10
  
n           : 121
Kappa           : 10
  
n           : 123
Kappa           : 10
  
n           : 125
Kappa           : 10
  
n           : 127
Kappa           : 10
  
n           : 128
Kappa           : 10
  
n           : 130
Kappa           : 10
  
n           : 132
Kappa           : 10
  
n           : 134
Kappa           : 10
  
n           : 136
Kappa           : 10
  
n           : 137
Kappa           

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_ratio_sousdet_100.pdf"

### Gain vs conditionnement

In [24]:
m = 100
ns = 100000
kappa_vals = vec(10 .^ range(0, 2, length=200))
gain = zeros(length(kappa_vals),length(ns))
for (j, n) in enumerate(ns)
    for (i, kappa) in enumerate(kappa_vals)
        A = badly_conditioned_underdetermined_matrix(m, n, kappa)
        x = randn(n)
        b = A * x
        res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
        x_prec, hist2 = lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        rationiter = hist2.niter/hist1.niter
        println("n           : $(n)")
        println("Kappa           : $(kappa)")
        println("  ")
        gain[i,j] = max(1 - rationiter,-1)
    end
end
plt =plot(kappa_vals, vec(gain);
    xscale=:log10,
    xlabel="Conditionnement",
    ylabel="Gain",
    color=:viridis,
    title="Gain en conditionnement",
)
savefig(plt,"Fig/Gain_VS_condi_sousdet_100-10000.png")
savefig(plt,"Fig/Gain_VS_condi_sousdet_100-10000.pdf")

n           : 100000
Kappa           : 1.0
  
n           : 100000
Kappa           : 1.0234114021054532
  
n           : 100000
Kappa           : 1.0473708979594496
  
n           : 100000
Kappa           : 1.0718913192051278
  
n           : 100000
Kappa           : 1.0969857978923836
  
n           : 100000
Kappa           : 1.1226677735108137
  
n           : 100000
Kappa           : 1.1489510001873091
  
n           : 100000
Kappa           : 1.1758495540521567
  
n           : 100000
Kappa           : 1.2033778407775895
  
n           : 100000
Kappa           : 1.2315506032928256
  
n           : 100000
Kappa           : 1.2603829296797275
  
n           : 100000
Kappa           : 1.2898902612533087
  
n           : 100000
Kappa           : 1.3200884008314178
  
n           : 100000
Kappa           : 1.3509935211980268
  
n           : 100000
Kappa           : 1.3826221737646558
  
n           : 100000
Kappa           : 1.414991297434576
  
n           : 100000
Kappa           : 1

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Gain_VS_condi_sousdet_100-10000.pdf"

# Parallélisme

In [25]:
ms = Int.([1000 10000 100000 1000000])
n = 1000
kappa = 70
threads_list = [1, 2, 4, 8, 16, 32]
times = zeros(length(threads_list),length(ms))
for (j,m) in enumerate(ms)
    for (i, t) in enumerate(threads_list)
        BLAS.set_num_threads(t) 
        println("Nombre de threads : $t")
        A = badly_conditioned_rectangular_matrix(m, n, kappa)
        x_true = randn(n)
        b = A * x_true

        lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)

        t_exec = @elapsed lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
        times[i,j] = t_exec
        println("Temps d'exécution : $(round(t_exec, digits=4)) sec")
    end
end
plt = plot(
    threads_list, times[:,1],
    marker=:o, lw=2,
    label="(m,n) = (1000,1000)",
    xlabel="Nombre de threads BLAS",
    ylabel="Temps d'exécution (s)",
    title="Performance en fonction des threads",
    legend=true
)
for i=2:length(ms)
    plot!(plt, threads_list, times[:,i],marker=:o,
        lw=2, label="(m,n) = ($(ms[i]),1000)")
end
savefig(plt,"Fig/Time_VS_Threads.png")
savefig(plt,"Fig/Time_VS_Threads.pdf")

Nombre de threads : 1
Temps d'exécution : 1.1743 sec
Nombre de threads : 2
Temps d'exécution : 0.6304 sec
Nombre de threads : 4
Temps d'exécution : 0.3657 sec
Nombre de threads : 8
Temps d'exécution : 0.4048 sec
Nombre de threads : 16
Temps d'exécution : 0.6996 sec
Nombre de threads : 32
Temps d'exécution : 2.4053 sec
Nombre de threads : 1
Temps d'exécution : 1.9316 sec
Nombre de threads : 2
Temps d'exécution : 1.2491 sec
Nombre de threads : 4
Temps d'exécution : 1.0038 sec
Nombre de threads : 8
Temps d'exécution : 1.0615 sec
Nombre de threads : 16
Temps d'exécution : 0.9913 sec
Nombre de threads : 32
Temps d'exécution : 3.194 sec
Nombre de threads : 1
Temps d'exécution : 12.5993 sec
Nombre de threads : 2
Temps d'exécution : 8.2183 sec
Nombre de threads : 4
Temps d'exécution : 5.1904 sec
Nombre de threads : 8
Temps d'exécution : 5.7208 sec
Nombre de threads : 16
Temps d'exécution : 5.1579 sec
Nombre de threads : 32
Temps d'exécution : 7.5216 sec
Nombre de threads : 1
Temps d'exécution 

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Time_VS_Threads.pdf"

In [28]:
plt = plot(
    threads_list, times[:,1],
    marker=:o, lw=2,
    label="(m,n) = (1000,1000)",
    xlabel="Nombre de threads BLAS",
    ylabel="Temps d'exécution (s)",
    title="Performance en fonction des threads",
    legend=true
)
for i=2:length(ms)
    plot!(plt, threads_list, times[:,i],marker=:o,
        lw=2, label="(m,n) = ($(ms[i]),1000)")
end
savefig(plt,"Fig/Time_VS_Threads.png")
savefig(plt,"Fig/Time_VS_Threads.pdf")

"c:\\Users\\ulric\\Projet_MTH8211\\Fig\\Time_VS_Threads.pdf"